In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.svm import SVC  
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import FastICA
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import decomposition
from sklearn.feature_selection import SelectPercentile as sp
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import RFECV

In [2]:
df = pd.read_csv('Network_dataset_19.csv')

In [3]:
df.head()

,ts,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,1556301245,192.168.1.32,42992.0,192.168.1.195,80.0,tcp,http,0.391745,270.0,467.0,...,0.0,0.0,-,-,-,-,-,-,1.0,password
1,1556301245,192.168.1.33,36108.0,52.59.177.21,21.0,tcp,-,3.034899,0.0,0.0,...,0.0,0.0,-,-,-,-,-,-,1.0,password
2,1556301245,192.168.1.32,43016.0,192.168.1.195,80.0,tcp,http,0.483543,221.0,1890.0,...,0.0,0.0,-,-,-,-,-,-,1.0,password
3,1556301245,192.168.1.32,43024.0,192.168.1.195,80.0,tcp,-,0.012174,0.0,0.0,...,0.0,0.0,-,-,-,-,-,-,1.0,password
4,1556301245,192.168.1.32,42982.0,192.168.1.195,80.0,tcp,http,0.323534,265.0,467.0,...,0.0,0.0,-,-,-,-,-,-,1.0,password


In [4]:
df = df.drop(['ts', 'src_ip', 'src_port', 'dst_ip', 'dst_port'], axis = 1)
df

,proto,service,duration,src_bytes,dst_bytes,conn_state,missed_bytes,src_pkts,src_ip_bytes,dst_pkts,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,tcp,http,0.391745,270.0,467.0,SF,0.0,6.0,530.0,5.0,...,0.0,0.0,-,-,-,-,-,-,1.0,password
1,tcp,-,3.034899,0.0,0.0,S0,0.0,3.0,180.0,0.0,...,0.0,0.0,-,-,-,-,-,-,1.0,password
2,tcp,http,0.483543,221.0,1890.0,SF,0.0,6.0,481.0,6.0,...,0.0,0.0,-,-,-,-,-,-,1.0,password
3,tcp,-,0.012174,0.0,0.0,SF,0.0,4.0,180.0,3.0,...,0.0,0.0,-,-,-,-,-,-,1.0,password
4,tcp,http,0.323534,265.0,467.0,SF,0.0,6.0,525.0,4.0,...,0.0,0.0,-,-,-,-,-,-,1.0,password
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639605,tcp,http,0.469347,221.0,1890.0,SF,0.0,6.0,481.0,6.0,...,0.0,0.0,-,-,-,-,-,-,1.0,password
639606,tcp,http,0.420843,263.0,467.0,SF,0.0,6.0,523.0,5.0,...,0.0,0.0,-,-,-,-,-,-,1.0,password
639607,tcp,http,0.009386,155.0,978.0,SF,0.0,5.0,375.0,4.0,...,0.0,0.0,-,-,-,-,-,-,1.0,password
639608,tcp,http,0.375529,263.0,467.0,SF,0.0,6.0,523.0,5.0,...,0.0,0.0,-,-,-,-,-,-,1.0,password


In [5]:
df = df.replace('-', np.NaN)

In [6]:
df.isnull().sum()

proto                          1
service                   145335
duration                       1
src_bytes                      1
dst_bytes                      1
conn_state                     1
missed_bytes                   1
src_pkts                       1
src_ip_bytes                   1
dst_pkts                       1
dst_ip_bytes                   1
dns_query                 637699
dns_qclass                     1
dns_qtype                      1
dns_rcode                      1
dns_AA                    637699
dns_RD                    637699
dns_RA                    637699
dns_rejected              637699
ssl_version               639610
ssl_cipher                639610
ssl_resumed               639610
ssl_established           639610
ssl_subject               639610
ssl_issuer                639610
http_trans_depth          637486
http_method               637486
http_uri                  637486
http_version              637504
http_request_body_len          1
http_respo

In [7]:
for i in df.columns:
    if df[i].isnull().sum() == 1: pass
    else: df = df.dropna(axis = 1)

KeyError: 'duration'

In [ ]:
df.isnull().sum()

In [ ]:
target = df.loc[:, ('type')]

In [ ]:
df = df.drop('type', axis = 1)

In [ ]:
cat = []
for i in df.columns:
    if len(df[i].unique()) <= 200: cat.append(i)
df_cat = df[cat]
df_cat

In [ ]:
df_temp = df.drop(df_cat, axis = 1)

In [ ]:
scaler = StandardScaler()
df[df_temp.columns] = scaler.fit_transform(df[df_temp.columns])

In [ ]:
df[df_temp.columns] = preprocessing.normalize(df[df_temp.columns])

In [ ]:
df

In [ ]:
label_encoder = preprocessing.LabelEncoder()
df['proto'] = label_encoder.fit_transform(df['proto'])
df['conn_state'] = label_encoder.fit_transform(df['conn_state'])

In [ ]:
def lr(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_lr = LogisticRegression()
    model_lr.fit(x_train, y_train)
    preds = model_lr.predict(x_test)
    return accuracy_score(y_test, preds)

In [ ]:
def dtc(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_dtc = DecisionTreeClassifier()   
    model_dtc.fit(x_train, y_train)
    preds = model_dtc.predict(x_test)
    return accuracy_score(y_test, preds)

In [ ]:
def rfc(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_rfc = RandomForestClassifier()    
    model_rfc.fit(x_train, y_train)
    preds = model_rfc.predict(x_test)
    return accuracy_score(y_test, preds)

In [ ]:
def knn(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_knn = KNeighborsClassifier()
    model_knn.fit(x_train, y_train)
    preds = model_knn.predict(x_test)
    return accuracy_score(y_test, preds)

In [ ]:
def svm(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_svm = SVC(kernel = 'sigmoid')
    model_svm.fit(x_train, y_train) 
    preds = model_svm.predict(x_test)
    return accuracy_score(y_test, preds)

In [ ]:
def nb(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_nb = GaussianNB()
    model_nb.fit(x_train, y_train)
    preds = model_nb.predict(x_test)
    return accuracy_score(y_test, preds)

In [ ]:
def xgbb(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_xgb = xgb.XGBClassifier()
    model_xgb.fit(x_train, y_train)
    preds = model_xgb.predict(x_test)
    return accuracy_score(y_test, preds)

In [ ]:
def adb(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_adb = AdaBoostClassifier()
    model_adb.fit(x_train, y_train)
    preds = model_adb.predict(x_test)
    return accuracy_score(y_test, preds)

In [ ]:
def mlp(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_mlp = MLPClassifier()    
    model_mlp.fit(x_train, y_train)
    preds = model_mlp.predict(x_test)
    return accuracy_score(y_test, preds)

In [ ]:
def gb(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_mlp = GradientBoostingClassifier() 
    model_mlp.fit(x_train, y_train)
    preds = model_mlp.predict(x_test)
    return accuracy_score(y_test, preds)

In [ ]:
target = label_encoder.fit_transform(target)

In [ ]:
x = df
y = target

In [ ]:
x.columns = x.columns.astype('str')

In [ ]:
#without feature selection


print('lr - ', lr(x, y))
print('dtc - ', dtc(x, y))
print('rfc - ', rfc(x, y))
print('knn - ', knn(x, y))
# print('svm - ', svm(x, y))
print('Naive Bayes - ', nb(x, y))
print('XGB - ', xgbb(x, y))
print('ADB - ', adb(x, y))
print('MLP - ', mlp(x, y))
print('GB - ', gb(x, y))

In [ ]:
def ica(x, i):
    ica = FastICA(n_components = i, random_state = 42)
    data_ica = ica.fit_transform(x)
    data_ica = pd.DataFrame(data_ica)
    return data_ica

In [ ]:
def pca(x, i):
    pca = decomposition.PCA(n_components = i, random_state=42)
    data_pca = pca.fit_transform(x)
    data_pca = pd.DataFrame(data_pca)
    return data_pca

In [ ]:
def mi(x, y):
    mi = sp(percentile = 50)
    data_mi = mi.fit(x, y)
    data_mi = mi.transform(x)
    data_mi = pd.DataFrame(data_mi)
    return data_mi

In [ ]:
def smf(x, y):
    lasso_model = Lasso(alpha = 0.01)
    selector = SelectFromModel(lasso_model)
    X_selected = selector.fit_transform(x, y)
    selected_features_indices = selector.get_support(indices=True)
    selected_features_names = x.columns[selected_features_indices]
    return x[selected_features_names]

In [ ]:
def sfs(x, y, model):
    sfs_selector = SequentialFeatureSelector(model, forward = True, k_features = len(x.columns) // 2)
    X_selected = sfs_selector.fit_transform(x, y)
    selected_features_names = list(x.columns[sfs_selector.k_feature_idx_])
    return x[selected_features_names]

In [ ]:
def rfe(x, y, model):
    rfecv_selector = RFECV(estimator = model, step = 1, cv = 5, scoring = 'accuracy')
    X_selected = rfecv_selector.fit_transform(x, y)
    selected_features_names = list(x.columns[rfecv_selector.support_])
    return x[selected_features_names]

In [ ]:
def cfs(x):
    corr_matrix = x.corr()
    high_corr_features = []
    
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > 0.8:
                colname = corr_matrix.columns[i]
                high_corr_features.append(colname)

    return x.drop(high_corr_features, axis=1)

In [ ]:
def fir(x, y):
    rfc = RandomForestClassifier(n_estimators=100)
    rfc.fit(x, y)
    feature_importances = list(zip(x.columns, rfc.feature_importances_))
    a = sorted(feature_importances, key=lambda x: x[1], reverse=True)
    ls = []
    for i in a[:len(a)//2 + 1]: ls.append(i[0])
    return x[ls]

In [ ]:
data_ica = ica(x, len(x.columns) // 2)

In [ ]:
data_pca = pca(x, len(x.columns) // 2)

In [ ]:
data_mi = mi(x, y)

In [ ]:
data_smf = smf(x, y)

In [ ]:
data_cfs = cfs(x)

In [ ]:
data_fir = fir(x, y)

In [ ]:
#with feature selection ica


print('lr - ', lr(data_ica, y))
print('dtc - ', dtc(data_ica, y))
print('rfc - ', rfc(data_ica, y))
print('knn - ', knn(data_ica, y))
# print('svm - ', svm(data_ica, y))
print('Naive Bayes - ', nb(data_ica, y))
print('XGB - ', xgbb(data_ica, y))
print('ADB - ', adb(data_ica, y))
print('MLP - ', mlp(data_ica, y))
print('GB - ', gb(data_ica, y))

In [ ]:
#with feature selection pca


print('lr - ', lr(data_pca, y))
print('dtc - ', dtc(data_pca, y))
print('rfc - ', rfc(data_pca, y))
print('knn - ', knn(data_pca, y))
# print('svm - ', svm(data_pca, y))
print('Naive Bayes - ', nb(data_pca, y))
print('XGB - ', xgbb(data_pca, y))
print('ADB - ', adb(data_pca, y))
print('MLP - ', mlp(data_pca, y))
print('GB - ', gb(data_pca, y))

In [ ]:
#with feature selection mi


print('lr - ', lr(data_mi, y))
print('dtc - ', dtc(data_mi, y))
print('rfc - ', rfc(data_mi, y))
print('knn - ', knn(data_mi, y))
# print('svm - ', svm(data_mi, y))
print('Naive Bayes - ', nb(data_mi, y))
print('XGB - ', xgbb(data_mi, y))
print('ADB - ', adb(data_mi, y))
print('MLP - ', mlp(data_mi, y))
print('GB - ', gb(data_mi, y))

In [ ]:
#with feature selection smf


print('lr - ', lr(data_smf, y))
print('dtc - ', dtc(data_smf, y))
print('rfc - ', rfc(data_smf, y))
print('knn - ', knn(data_smf, y))
# print('svm - ', svm(data_smf, y))
print('Naive Bayes - ', nb(data_smf, y))
print('XGB - ', xgbb(data_smf, y))
print('ADB - ', adb(data_smf, y))
print('MLP - ', mlp(data_smf, y))
print('GB - ', gb(data_smf, y))

In [ ]:
#with feature selection csf


print('lr - ', lr(data_cfs, y))
print('dtc - ', dtc(data_cfs, y))
print('rfc - ', rfc(data_cfs, y))
print('knn - ', knn(data_cfs, y))
# print('svm - ', svm(data_cfs, y))
print('Naive Bayes - ', nb(data_cfs, y))
print('XGB - ', xgbb(data_cfs, y))
print('ADB - ', adb(data_cfs, y))
print('MLP - ', mlp(data_cfs, y))
print('GB - ', gb(data_cfs, y))

In [ ]:
#with feature selection fir


print('lr - ', lr(data_fir, y))
print('dtc - ', dtc(data_fir, y))
print('rfc - ', rfc(data_fir, y))
print('knn - ', knn(data_fir, y))
# print('svm - ', svm(data_fir, y))
print('Naive Bayes - ', nb(data_fir, y))
print('XGB - ', xgbb(data_fir, y))
print('ADB - ', adb(data_fir, y))
print('MLP - ', mlp(data_fir, y))
print('GB - ', gb(data_fir, y))